In [4]:
import folium
import numpy as np
import pickle
from math import radians, cos, sin, asin, sqrt

# ---------------------------------------------------------
# HELPER: Calculate Distance (Haversine)
# ---------------------------------------------------------
def haversine(lon1, lat1, lon2, lat2):
    """Calculates distance in km between two lat/lon points"""
    R = 6371  # Earth radius in km
    dlat, dlon = radians(lat2 - lat1), radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    return R * c

def generate_interactive_map():
    # 1. SETUP COORDINATES
    # -----------------------------------------------------
    # Mirissa Surf Spot (Actual Location)
    SPOT_NAME = "Mirissa Point"
    SPOT_LAT, SPOT_LON = 5.942, 80.456

    # ERA5 "Virtual Buoy" (Nearest Ocean Point from your logic)
    # Based on your previous data, let's assume this is the nearest valid point
    BUOY_LAT, BUOY_LON = 5.75, 80.50 

    # Load Grid Data (or synthesize if missing)
    try:
        with open("model_metadata.pkl", "rb") as f:
            meta = pickle.load(f)
        grid_lats = meta['latitude']
        grid_lons = meta['longitude']
    except FileNotFoundError:
        print("⚠️ Metadata not found. Using synthetic grid for demo.")
        grid_lats = np.arange(5.0, 6.5, 0.25)
        grid_lons = np.arange(80.0, 81.0, 0.25)

    # Calculate gap distance
    dist_km = haversine(SPOT_LON, SPOT_LAT, BUOY_LON, BUOY_LAT)

    # 2. INITIALIZE MAP
    # -----------------------------------------------------
    # Center map between the two points
    center_lat = (SPOT_LAT + BUOY_LAT) / 2
    center_lon = (SPOT_LON + BUOY_LON) / 2

    m = folium.Map(
        location=[center_lat, center_lon], 
        zoom_start=11,
        control_scale=True,
        tiles=None # We will add custom tiles
    )

    # 3. ADD BASE LAYERS (Satellite vs Street)
    # -----------------------------------------------------
    # Layer 1: Satellite (Esri World Imagery) - BEST for Surf Projects
    folium.TileLayer(
        tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr='Esri',
        name='Satellite (Esri)',
        overlay=False,
        control=True
    ).add_to(m)

    # Layer 2: OpenStreetMap (Standard)
    folium.TileLayer(
        tiles='OpenStreetMap',
        name='Street Map',
        overlay=False,
        control=True
    ).add_to(m)

    # 4. DRAW THE ERA5 GRID
    # -----------------------------------------------------
    # We filter points to only show those nearby to avoid clutter
    grid_group = folium.FeatureGroup(name="ERA5 Grid Points (0.25°)")
    
    for lat in grid_lats:
        for lon in grid_lons:
            # Only plot if within view range
            if 5.5 < lat < 6.2 and 80.2 < lon < 80.8:
                folium.CircleMarker(
                    location=[lat, lon],
                    radius=3,
                    color='white',
                    weight=1,
                    fill=True,
                    fill_color='black',
                    fill_opacity=0.6,
                    popup=f"ERA5 Node<br>Lat: {lat}<br>Lon: {lon}"
                ).add_to(grid_group)
    
    grid_group.add_to(m)

    # 5. MARKERS & CONNECTORS
    # -----------------------------------------------------
    
    # A. The Surf Spot (Red Marker)
    folium.Marker(
        [SPOT_LAT, SPOT_LON],
        popup=f"<b>{SPOT_NAME}</b><br>Target Location",
        icon=folium.Icon(color='red', icon='flag', prefix='fa')
    ).add_to(m)

    # B. The Virtual Buoy (Blue Marker)
    folium.Marker(
        [BUOY_LAT, BUOY_LON],
        popup=f"<b>Virtual Buoy</b><br>Nearest ERA5 Data Source<br>Lat: {BUOY_LAT}<br>Lon: {BUOY_LON}",
        icon=folium.Icon(color='blue', icon='anchor', prefix='fa')
    ).add_to(m)

    # C. The Connection Line (The "Gap")
    line = folium.PolyLine(
        locations=[[SPOT_LAT, SPOT_LON], [BUOY_LAT, BUOY_LON]],
        weight=3,
        color='orange',
        dash_array='10',
        tooltip=f"Interpolation Gap: {dist_km:.2f} km"
    ).add_to(m)

    # 6. FINALIZE
    # -----------------------------------------------------
    folium.LayerControl().add_to(m)
    
    # Save
    output_file = "interactive_gap_map.html"
    m.save(output_file)
    print(f"✅ Map generated: {output_file}")
    print("   Open this file in your browser to interact.")

if __name__ == "__main__":
    generate_interactive_map()

✅ Map generated: interactive_gap_map.html
   Open this file in your browser to interact.
